In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator


2023-07-04 19:19:55.713902: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-04 19:19:55.824694: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-04 19:19:55.826843: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 19:19:57.827699: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Load the MNIST dataset

In [2]:
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()
x_train = x_train/255.0
x_test = x_test/255.0


11490434/11490434 [==============================] - 8s 1us/step


# **Data Augmentation function**

In [3]:
datagen = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.1,
    width_shift_range = 0.1,
    height_shift_range= 0.1,
)

# Reshape the tainging and testing data and augmentation

In [4]:
datagen.fit(x_train.reshape((-1,28,28,1)))
augmented_train = datagen.flow(x_train.reshape((-1,28,28,1)),y_train, batch_size=32)

datagen.fit(x_test.reshape((-1,28,28,1)))
augmented_test = datagen.flow(x_test.reshape((-1,28,28,1)),y_test,batch_size = 32)

# **Create the CNN model and Compilation**

1.   Two convo2d layer with (3,3) filter,and maxpolling (2,2), for feature extraction 
2.   Two dense layer with 64 and 10 neuron,**for classification**


In [5]:
from keras.layers.serialization import activation
model = keras.Sequential([
    #feature extraction part
    keras.layers.Conv2D(32,(3,3),activation='relu',padding='same',input_shape=(28,28,1)),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Conv2D(64,(3,3),padding='same',activation='relu'),
    keras.layers.MaxPooling2D((2,2)),
    
    #classification part
    keras.layers.Flatten(),
    keras.layers.Dense(128,activation='relu'),#hidden layer with 128 neurons
    keras.layers.Dense(64,activation='relu'),#hidden layer with 64 neurons
    keras.layers.Dense(32,activation='relu'),#hidden layer with 32 neurons
    keras.layers.Dense(10,activation='softmax')#output layer with 10 neurons
    ])


model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy']
)

2023-07-04 19:20:34.098496: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-04 19:20:34.098647: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: rafz
2023-07-04 19:20:34.098711: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: rafz
2023-07-04 19:20:34.098965: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 525.125.6
2023-07-04 19:20:34.099067: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.125.6
2023-07-04 19:20:34.099090: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:309] kernel version seems to match DSO: 525.125.6


# **Create the ANN model and Compilation**


In [14]:
model2 = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),#input layer
    keras.layers.Dense(128,activation='relu'),#hidden layer with 128 neurons
    keras.layers.Dense(64,activation='relu'),#hidden layer with 64 neurons
    keras.layers.Dense(32,activation='relu'),#hidden layer with 32 neurons
    keras.layers.Dense(10,activation='softmax')#output layer with 10 neurons
])

model2.compile( #model compilation
    optimizer = 'adam',
    loss='sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

# Train the CNN models

In [6]:
model.fit(augmented_train,epochs=20)

Epoch 1/20


2023-07-04 19:20:56.691038: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1875/1875 [==============================] - 67s 35ms/step - loss: 0.2555 - accuracy: 0.9182
Epoch 2/20
1875/1875 [==============================] - 64s 34ms/step - loss: 0.0914 - accuracy: 0.9721
Epoch 3/20
1875/1875 [==============================] - 64s 34ms/step - loss: 0.0682 - accuracy: 0.9794
Epoch 4/20
1875/1875 [==============================] - 64s 34ms/step - loss: 0.0572 - accuracy: 0.9817
Epoch 5/20
1875/1875 [==============================] - 72s 38ms/step - loss: 0.0512 - accuracy: 0.9841
Epoch 6/20
1875/1875 [==============================] - 65s 35ms/step - loss: 0.0472 - accuracy: 0.9859
Epoch 7/20
1875/1875 [==============================] - 56s 30ms/step - loss: 0.0416 - accuracy: 0.9876
Epoch 8/20
1875/1875 [==============================] - 57s 30ms/step - loss: 0.0396 - accuracy: 0.9877
Epoch 9/20
1875/1875 [==============================] - 55s 30ms/step - loss: 0.0367 - accuracy: 0.9887
Epoch 10/20
1875/1875 [==============================] - 55s 29ms/step - lo

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense (Dense)               (None, 128)               4

# Train the ANN model

In [11]:
model2.fit(augmented_train,epochs=20)

Epoch 1/20
1875/1875 [==============================] - 25s 14ms/step - loss: 0.0848 - accuracy: 0.9737
Epoch 2/20
1875/1875 [==============================] - 25s 14ms/step - loss: 0.0844 - accuracy: 0.9739
Epoch 3/20
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0851 - accuracy: 0.9736
Epoch 4/20
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0816 - accuracy: 0.9748
Epoch 5/20
1875/1875 [==============================] - 23s 13ms/step - loss: 0.0804 - accuracy: 0.9748
Epoch 6/20
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0817 - accuracy: 0.9747
Epoch 7/20
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0776 - accuracy: 0.9766
Epoch 8/20
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0763 - accuracy: 0.9759
Epoch 9/20
1875/1875 [==============================] - 24s 13ms/step - loss: 0.0760 - accuracy: 0.9771
Epoch 10/20
1875/1875 [==============================] - 24s 13m

In [13]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               100480    
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 10)                330       
                                                                 
Total params: 111,146
Trainable params: 111,146
Non-trainable params: 0
_________________________________________________________________


# Evaluation difference **ANN & CNN** 


In [9]:

print("ANN model evaluation")
model2.evaluate(augmented_test)
print("\n\n\nCNN model evaluation")
model.evaluate(augmented_test)




CNN model evaluation


2023-07-04 19:46:26.041146: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


313/313 [==============================] - 7s 21ms/step - loss: 0.0329 - accuracy: 0.9902


[0.032897233963012695, 0.9901999831199646]